In [9]:
# %pip install numpy

In [10]:
import numpy as np

In [11]:
def calc_avg_naive(input: np.ndarray) -> np.float64:
    result = np.float64(0)
    for idx in range(input.shape[0]):
        result += input[idx] / input.shape[0]
    return result

In [12]:
def calc_avg_v2(input: np.ndarray) -> np.float64:
    input = np.sort(input)
    result = np.float64(0)
    for idx in range(input.shape[0]):
        result += input[idx] / input.shape[0]
    return result

In [13]:
def calc_avg_v3(iter: np.nditer) -> np.float64:
    avg = 0
    cnt = 0
    for val in iter:
        avg = avg * cnt/(cnt+1) + val/(cnt+1)
        cnt += 1
    return avg

In [14]:
def make_test_array(big_val, big_rep, small_val, small_rep):
    x1 = np.full(big_rep, fill_value=np.float64(big_val))
    x2 = np.full(small_rep, fill_value=np.float64(small_val))
    x = np.concat([x1, x2])
    return x

In [15]:
x = make_test_array(big_val=10**20, big_rep=1, small_val=1.0, small_rep=10**6-1)
x.shape

(1000000,)

[2sum algo](https://en.wikipedia.org/wiki/2Sum):

In [18]:
import typing as t

def two_sum(a: float, b: float) -> t.Tuple[float, float]:
    s = a + b
    a_virt = s - b
    b_virt = s - a_virt
    ea = a - a_virt
    eb = b - b_virt
    e = ea + eb
    return s, e

In [19]:
a = 10**30
b = 1.0
two_sum(a,b)

(1e+30, 1.0)

In [20]:
def fast2sum(a: float, b: float, swap: bool = True) -> t.Tuple[float, float]:
    if abs(a) < abs(b) and swap:
        a, b = b, a
    s = a + b
    b2 = s - a
    e = b - b2
    return s, e

In [21]:
fast2sum(a, b, swap=False)

(1e+30, 1.0)

In [22]:
fast2sum(b, a, swap=False)

(1e+30, 0.0)

In [23]:
fast2sum(b, a, swap=True)

(1e+30, 1.0)

$$\bar X_N = \frac{1}{N} \sum_{i=1}^N X_i$$
$$\bar X_{N} = \frac {(N-1)} {N} \times \bar X_{N-1} + \frac 1 N \times {X_N} $$
$$\bar X_{N} = (\frac {N} {N} \times \bar X_{N-1} - \frac 1 N \times \bar X_{N-1}) + \frac 1 N \times {X_N} $$
$$\bar X_{N} = \bar X_{N-1} + \frac {X_N - \bar X_{N-1}} {N}$$

In [24]:
def calc_avg_kahan_welford(iter: np.nditer, callback: t.Optional[t.Callable[[float, float], None]] = None, call_freq: int = 1) -> [float, float]:
    result = error = 0.0
    count = 0
    call_ctr = call_freq
    for value in iter:
        x1, x2 = result * count/(count+1), value / (count+1)
        result, error_part = fast2sum(x1, x2)
        error += error_part
        count += 1
        if callback:
            call_ctr -= 1
            if call_ctr <= 0:
                callback(result, error)
                call_ctr = call_freq
    return result, error

In [25]:
def calc_avg_kahan_welford_v2(
    iter: np.nditer, 
    callback: t.Optional[t.Callable[[float, float], None]] = None, 
    call_freq: int = 1
) -> [float, float]:
    result = error = 0.0
    count = 0
    call_ctr = call_freq
    for value in iter:
        count += 1
        delta = (value - result) / count
        result, error = fast2sum(result, delta+error, swap=True)
        if callback:
            call_ctr -= 1
            if call_ctr <= 0:
                callback(result, error)
                call_ctr = call_freq
    return result, error

In [40]:
x = make_test_array(big_val=10**20, big_rep=1, small_val=1.0, small_rep=10**6-1)

In [41]:
calc_avg_naive(x)

np.float64(100000000000000.0)

In [42]:
calc_avg_v2(x)

np.float64(100000000000001.0)

In [43]:
calc_avg_v3(np.nditer(x))

np.float64(99999999999999.77)

In [44]:
calc_avg_kahan_welford(
    np.nditer(x), 
    callback = lambda avg,err: print(f"{avg=}, {err=}"),
    call_freq = len(x) / 10
)

avg=np.float64(1000000000000013.1), err=np.float64(11.09014612986337)
avg=np.float64(500000000000013.1), err=np.float64(11.783290810429856)
avg=np.float64(333333333333343.1), err=np.float64(12.188755085205699)
avg=np.float64(250000000000005.2), err=np.float64(12.476436740991062)
avg=np.float64(200000000000004.16), err=np.float64(12.699580042305662)
avg=np.float64(166666666666669.03), err=np.float64(12.881901432432912)
avg=np.float64(142857142857141.7), err=np.float64(13.03605199321237)
avg=np.float64(124999999999999.72), err=np.float64(13.169583296550895)
avg=np.float64(111111111111110.86), err=np.float64(13.287366262762905)
avg=np.float64(99999999999999.77), err=np.float64(13.392726722865024)


(np.float64(99999999999999.77), np.float64(13.392726722865024))

In [45]:
calc_avg_kahan_welford_v2(
    np.nditer(x), 
    callback = lambda avg,err: print(f"{avg=:.6f}, {err=:.6f}"),
    call_freq = len(x) / 10
)

avg=1000000000000001.000000, err=-0.019924
avg=500000000000001.000000, err=-0.010029
avg=333333333333334.312500, err=0.014098
avg=250000000000001.000000, err=-0.005062
avg=200000000000001.000000, err=-0.004056
avg=166666666666667.656250, err=0.007034
avg=142857142857143.843750, err=0.010490
avg=125000000000001.000000, err=-0.002541
avg=111111111111112.109375, err=-0.000520
avg=100000000000001.000000, err=-0.002030


(np.float64(100000000000001.0), np.float64(-0.002029985189437866))

In [37]:
len(x)

1000000

In [ ]:
10**20 / 10000000

In [ ]:
(len(x) - 1)

In [ ]:
10**20/10000000 + 9999999/10000000

In [ ]:
two_sum(10**20/10000000, 9999999/10000000)

In [ ]:
fast2sum(10**20/10000000, 9999999/10000000)

In [ ]:
1 - (9999999 / 10000000)

In [ ]:
sum(int(item) for item in x.tolist()) / len(x)

In [ ]:
x.astype('int').sum()/len(x)

In [ ]:
((2 * 10**-30 + 10**30) - 10**30) - 10**-30

In [ ]:
class DDouble:
    def __init__(self, val: float = 0.0, err: float = 0.0):
        self.val = val
        self.err = err

    def __add__(self, y: "DDouble") -> "DDouble":
        self.val, err = two_sum(self.val, y.val)
        self.err += err
        self.err += y.err
        return self
    
    def __sub__(self, y: "DDouble") -> "DDouble":
        return self.__add__(DDouble(val=-y.val, err=-y.err))
    
    def __repr__(self) -> str:
        return f"({self.val}, {self.err})"

In [ ]:
((DDouble(2*10**-30) + DDouble(10**30)) - DDouble(10**30)) - DDouble(10**-30)

In [ ]:
two_sum(2 * 10**-30, 10**30)

In [ ]:
from decimal import Decimal

In [ ]:
x1 = Decimal(10.0**20)
x1.as_tuple()

In [ ]:
x2 = Decimal(1.0)
x2.as_tuple()

In [ ]:
10.0**20 + 1.0

In [ ]:
x1 + x2

In [ ]:
Decimal(0.1).as_tuple()

In [ ]:
Decimal(154.3).as_tuple()